In [19]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict

In [20]:
OSM_FILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
#OSMFILE = "san-francisco-bay_california.osm"

In [21]:
def count_tags(filename):
    tags = defaultdict(int)
        
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        tags[elem.tag] += 1
        
        # TODO(hkwik): I know I ought to clear the root, but not sure about elem.tag in tags.
        # Once I understand, I'll put this in.
        # if event == 'end' and elem.tag in tags: 
        #    root.clear()
    
    return tags

count_tags(OSM_FILE)

defaultdict(int,
            {'member': 2866,
             'nd': 290588,
             'node': 245418,
             'osm': 2,
             'relation': 220,
             'tag': 110246,
             'way': 25960})

In [22]:
expected = ["Alley", "Avenue", "Boulevard", "Circle", "Common", "Commons", 
            "Corte", "Court", "Courtyard", "Drive", "Expressway", 
            "Highway", "Lane", "Parkway", "Place", "Plaza",
            "Real", "Road", "Square", "Street", "Terrace", "Trail",
            "Way"]

mapping = { "Aly": "Alley"
            "AVE": "Avenue",
            "Ave": "Avenue",
            "Aveenue": "Avenue",
            "Avenie": "Avenue",
            "Ave.": "Avenue",
            "BLVD": "Boulevard",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Boulvevard": "Boulevard",
            "Boulevar": "Boulevard",
            "Cir": "Circle",
            "Circle:": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Ctr": "Center",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "St": "Street",
            "St.": "Street"
            
            }

In [25]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                    
        if event == 'end' and elem.tag in tags: 
            root.clear()
                    
    osm_file.close()
    return street_types


def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        new_street_type = mapping[street_type]
        name = re.sub(street_type_re, new_street_type, name)
    return name

In [24]:
audit(OSMFILE)


defaultdict(set,
            {'0.1': {'Ala 680 PM 0.1'},
             '1': {'10795 Hwy 1',
              '2030 Hwy 1',
              'California Highway 16, House No. 1',
              'Old Bernal Ave #1',
              'Prospect Rd #1',
              'State Hwy 1',
              'Stewart Drive Suite #1',
              'W Of Us 101 @ Jct Sr 1'},
             '10': {'South St #10'},
             '100': {'Northgate Blvd #100', 'Woodside Road, Suite 100'},
             '101': {'Highway 101',
              'Nw Quad Lincoln Ave / Us 101',
              'Se Quad Smith Ranch Rd / Us 101'},
             '102': {'County Road 102'},
             '105': {'N California Blvd #105'},
             '10675': {'10675'},
             '11': {'Fairview Rd #11'},
             '110': {'West Angela Street, Suite 110'},
             '110,': {'Promenade Circle #110,'},
             '114': {'West Evelyn Avenue Suite #114'},
             '116': {'Hwy 116',
              'In "Y" Jct Of Sr 121 / Sr 116',
          